# Experiment Runner: HLE Analysis with DeepSeek

このノートブックは設定ファイルに基づいて実験プロセス全体をオーケストレーションします

## 1. Setup and Configuration

In [1]:
import os
from pathlib import Path

# 現在の作業ディレクトリを、プロジェクトのルートに設定する（Colabと同じ）

# もし現在のディレクトリ名が 'notebooks' なら、一つ上の階層（プロジェクトルート）に移動する
if Path.cwd().name == 'notebooks':
    os.chdir('..')

print(f"✅ 作業ディレクトリをプロジェクトルートに設定しました: {Path.cwd()}")

✅ 作業ディレクトリをプロジェクトルートに設定しました: /app


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む
load_dotenv('.env')

hf_token_check = os.getenv('HUGGINGFACE_API_KEY')
print(f"Hugging Face Token Loaded: {'Yes' if hf_token_check else 'No'}")
if hf_token_check:
    # トークン全体は表示せず、最初の数文字だけ表示してトークンが正しく読み込まれたか確認する
    print(f"Token starts with: {hf_token_check[:5]}...") 

from utils import load_config, load_prompt_template, ensure_dir
from data_loader import load_hle_dataset
from model_handler import ModelHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Hugging Face Token Loaded: Yes
Token starts with: hf_ag...


In [ ]:
# 実験したい設定ファイルを選択する
CONFIG_FILE = 'configs/deepseek_base.yml'

print(f"Loading configuration from: {CONFIG_FILE}")
config = load_config(CONFIG_FILE)

# promptsをロード
mc_prompt_template = load_prompt_template('mc_system_prompt')
em_prompt_template = load_prompt_template('em_system_prompt')

print("Configuration and prompts loaded successfully.")
# configの内容を表示
print("Configuration:", config)

Loading configuration from: configs/deepseek_base.yml
Configuration and prompts loaded successfully.
Configuration: {'MODEL_NAME': 'deepseek-ai/DeepSeek-R1-0528-Qwen3-8B', 'DATASET_NAME': 'cais/hle', 'output_path': 'llm-hle-experiment/outputs/results', 'NUM_SAMPLES_TO_RUN': 5, 'MAX_LENGTH': 4096, 'TEMPERATURE': 0.0}


## 2. Load Dataset

In [ ]:
dataset = load_hle_dataset(config['DATASET_NAME'])

# 開発時はサンプル数を制限する
num_samples = config.get('NUM_SAMPLES_TO_RUN', len(dataset))
subset_dataset = dataset.select(range(num_samples))

print(f"Dataset loaded. Running on {len(subset_dataset)} samples.")
display(subset_dataset)

Loading dataset: cais/hle (split: test)
Original dataset size: 2500
Filtered to text-only dataset size: 2158
Dataset loaded. Running on 5 samples.


Dataset({
    features: ['id', 'question', 'image', 'image_preview', 'answer', 'answer_type', 'author_name', 'rationale', 'rationale_image', 'raw_subject', 'category', 'canary'],
    num_rows: 5
})

## 3. Load Model and Tokenizer

In [7]:
model_handler = ModelHandler(
    model_name=config['MODEL_NAME'],
    hf_token=os.getenv('HUGGINGFACE_API_KEY')
)

Initializing ModelHandler on device: cpu
Loading tokenizer: deepseek-ai/DeepSeek-R1-0528-Qwen3-8B
Loading model: deepseek-ai/DeepSeek-R1-0528-Qwen3-8B


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


Model and tokenizer loaded successfully.


## 4. Run Inference Loop

In [ ]:
results = []

for example in tqdm(subset_dataset):
    if example['answer_type'] == 'multiple_choice': # Multiple Choice
        prompt_template = mc_prompt_template
    else: # Exact Match
        prompt_template = em_prompt_template
    final_prompt = prompt_template.format(question=example['question'])
    
    # モデルからの応答を生成
    raw_output = model_handler.generate(
        prompt=final_prompt, 
        max_length=config['MAX_LENGTH'], 
        temperature=config['TEMPERATURE']
    )
    
    # 応答を解析
    parsed_data = model_handler.parse_output(raw_output)
    
    # 結果を辞書形式で保存
    record = {
        "id": example['id'],
        "question": example['question'],
        "ground_truth": example['answer'],
        "model_answer": parsed_data.get('answer'),
        "confidence": parsed_data.get('confidence'),
        "explanation": parsed_data.get('explanation'),
        "think_process": parsed_data.get('think_process'),
        "parse_error": parsed_data.get('error'),
        "raw_output": raw_output
    }
    results.append(record)


  0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


## 5. Save Results

In [ ]:
results_df = pd.DataFrame(results)

# 出力ディレクトリとファイル名を設定
output_dir = config['output_path']
config_name = CONFIG_FILE.split('/')[-1].replace('.yml', '')
output_csv_path = f"{output_dir}/{config_name}_results.csv"

# 出力ディレクトリを確認し、存在しない場合は作成する
ensure_dir(output_dir)

# 結果をCSVファイルに保存
results_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"Results successfully saved to: {output_csv_path}")
display(results_df.head())